In [1]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
url_re = re.compile("https?://[^\s]+")
mention_re = re.compile("@(\w+)")
def preprocessing(text):
    text_clean = url_re.sub("<url>",text)
    text_clean = mention_re.sub("<mention>", text_clean)
    text_clean = text_clean.lower()
    return text_clean 

In [3]:
def representationBOW(corpus):
    vectorizerTrain = TfidfVectorizer(ngram_range = (3,5),max_df=0.95,min_df=2,analyzer="char_wb")
    bow = vectorizerTrain.fit_transform(corpus)
    return bow,vectorizerTrain

In [4]:
import csv
dataText = []
dataLabels = []
i = 0
with open('Tweets.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    for row in csv_reader:
        if i >0:
            dataText.append(preprocessing(row[10]))
            dataLabels.append(row[1])
        i+=1

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataText, dataLabels, test_size=0.20, random_state=42)

In [6]:
bowTrain,vectorizer = representationBOW(X_train,character=True)

/Users/marcosesteve/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [7]:
bowTest = vectorizer.transform(X_test)

/Users/marcosesteve/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [8]:
from sklearn.svm import SVC
kernels = ["rbf","linear"]
cs = [1,10,100,1000]
for kernel in kernels:
    for c in cs:
        clf = SVC(kernel=kernel,C=c)
        clf.fit(bowTrain, y_train)
        print("%s \t %d \t %.3f " %(kernel[0:3],c,clf.score(bowTest,y_test)))

rbf 	 1 	 0.645 
rbf 	 10 	 0.645 
rbf 	 100 	 0.645 
rbf 	 1000 	 0.683 
lin 	 1 	 0.815 
lin 	 10 	 0.788 
lin 	 100 	 0.771 
lin 	 1000 	 0.761 


In [13]:
from sklearn.linear_model import LogisticRegression
cs = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 10, 100, 1000]
for c in cs:
    clf = LogisticRegression(C=c)
    clf.fit(bowTrain, y_train)
    print("%.3f \t %.3f " %(c,clf.score(bowTest,y_test)))

0.050 	 0.682 
0.100 	 0.717 
0.250 	 0.762 
0.500 	 0.798 
0.750 	 0.810 
1.000 	 0.813 
10.000 	 0.812 
100.000 	 0.796 
1000.000 	 0.781 
